# Bielik + Ollama – czyli proste uruchomienie LLM

LLM można używać bezpośrednio korzystając z biblioteki **transformers** od HuggigFace, lecz na dłuższą metę może okazać się to uciążliwe – np. trzymanie modelu w obiektach i zarządzanie nimi (często powiązane z obsługą GPU).

Z pomocą przychodzą różne narzędzia takie jak: **Ollama**, **vLLM**, **Text Generation Interface (TGI)** czy **llama.cpp** – w tym tutorialu ze względu na prostotę zostanie omówione pierwsze rozwiązanie.

## Ollama

Ollama to narzędzie do uruchamiania i zarządzania dużymi modelami językowymi na lokalnym komputerze. Zapewnia prosty interfejs do korzystania z gotowych modeli, jak również możliwość ich personalizacji i dostosowywania za pomocą specjalnych plików konfiguracyjnych. Ollama jest dostępna na różnych platformach, w tym macOS, Windows oraz Linux.

Przydatne linki:
  - Oficjalna strona projektu: https://ollama.com
  - Github: https://github.com/ollama/ollama
  - Dokumentacja API: https://github.com/ollama/ollama/blob/main/docs/api.md

### Ollama instalacja

Narzędzie jest wspierana na platformach macOS, Windows oraz Linux. Należy zainstalować ją jako klasyczną aplikację. Na platformach Windows i macOS będzie to możliwe poprzez klasyczne pobranie z zainstalowanie pakietu, natomiast dla Linux będzie trzeba odpalić specjalną komendę *[stan na sierpień 2024]*.

Szczegółowe informacje znajdują się na stronie projektu.

Aplikacja działa w oparciu o Command Line Interface (CLI) – wiersz poleceń. W celu weryfikacji poprawnej instalacji zalecam uruchomienie komendy: `ollama`. Jako wynik powinno pokazać się wynik przedstawiony poniżej (dokładnie instrukcja tego narzędzia). Jeśli pojawią się błędy to znaczy że narzędzie nie zostało zainstalowane poprawnie.

```shell
(base) sample_user@MacBook-Pro SPEAKLEASH % ollama
Large language model runner

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.
```

### Ollama uruchomienie poprzez CLI

Aby uruchomić LLM należy wpisać komendę:

```shell
ollama run SpeakLeash/bielik-7b-instruct-v0.1-gguf
```

Za pierwszym razem model będzie się pobierać na komputer co może potrwać dłuższą chwilę.
Po poprawnym załadowaniu będzie można zacząć korzystać z modelu w konsoli.

```shell
(base) marcinwatroba@DataScienceServer:~$ ollama run SpeakLeash/bielik-7b-instruct-v0.1-gguf
>>> Ile to jest 2+4?
<s>  2 plus 4 równa się 6.

>>> Kto napisał Pana Tadeusza?
<s>  Adam Mickiewicz napisał "Pana Tadeusza".

>>>
```

### Ollama – korzystanie z API (curl)

Ollama poza korzystaniem z CLI udostępnia również możliwość komunikacji poprzez Rest API.
Pozwala to na korzystanie z modelu w praktycznie każdej aplikacji, co daje całkiem spore możliwości.

Domyślnie aplikacja działa na porcie 11434. Oznacza to że jej domyślny host to http://localhost:11434 i z niego będziemy korzystać w poniższych przykładach.

Najprostszym przykładem będzie po prostu uruchomienie klasycznego zapytania dla zapytania składającego się z listy wiadomości (szczegóły dotyczące API znajdują się pod adresem https://github.com/ollama/ollama/blob/main/docs/api.md).

Poniżej znajduje się polecenie zapytanie w popularnej bibliotece curl -- można jej używać w formie CLI (bez większych umiejętności programowania)

```shell
curl http://localhost:11434/api/chat -d '{
  "model": "SpeakLeash/bielik-7b-instruct-v0.1-gguf",
  "stream": false,
  "messages": [
    {
      "role": "system",
      "content": "Odpowiedz krótko na pytanie"
    },
    {
      "role": "user",
      "content": "Kim jest Adam Mickiewicz?"
    }
  ]
}'
```

Zapytanie ma body w formie JSON, gdzie istotne są 3 parametry: `model`, `stream` i `messages`.
W odpowiedzi otrzymany następująxcy obiekt:
```
{
   "model":"SpeakLeash/bielik-7b-instruct-v0.1-gguf",
   "created_at":"2024-08-21T15:45:22.580717683Z",
   "message":{
      "role":"assistant",
      "content":"\u003cs\u003e  Adam Mickiewicz to polski poeta, uważany za jednego z najważniejszych twórców literatury romantyzmu. Znany m.in. z dzieł \"Pan Tadeusz\" i \"Dziady\"."
   },
   "done_reason":"stop",
   "done":true,
   "total_duration":967596312,
   "load_duration":4041118,
   "prompt_eval_count":44,
   "prompt_eval_duration":34273000,
   "eval_count":65,
   "eval_duration":798981000
}
```

### Ollama – korzystanie z API (python)

Analogiczny przykład z powyższego zapytania można wykonać w środowisku python.
W tym celu należy zainstalować wymagane zależności oraz napisać kod jaki będzie wykonywał zapytanie.

Poniżej znajduje się skrypt instalujący biblioteki `httpx` i `requests`. Są one alternatywne i dają takie same funkcjonalności -- jeśli nie znasz żadnej wybierz `httpx`.

In [ ]:
!pip install -qq httpx requests

Do celów testowych zastosowano nazwę domeny 'localhost'. Można również zastosować adres IP wskazujący na serwer ollamy.

In [ ]:
# pprint jest dodatkowe i można użyć zwykłego print, który nie formatuje składni
from pprint import pprint

# Stałe dla zapytania — będą tak samo wykorzystane w requests i httpx
URL = "http://localhost:11434/api/chat"
PAYLOAD = {
    "model": "SpeakLeash/bielik-7b-instruct-v0.1-gguf",
    "stream": False,
    "messages": [
        {
            "role": "system",
            "content": "Odpowiedz krótko na pytanie"
        },
        {
            "role": "user",
            "content": "Kim jest Adam Mickiewicz?"
        }
    ]
}

In [ ]:
import httpx

response = httpx.post(URL, json=PAYLOAD, timeout=500)
pprint(response.json())

{'created_at': '2024-08-22T15:50:21.438020749Z',
 'done': True,
 'done_reason': 'stop',
 'eval_count': 75,
 'eval_duration': 646169000,
 'load_duration': 1891913891,
 'message': {'content': ' Adam Mickiewicz to polski poeta, który brał udział w '
                        'powstaniu listopadowym i uważany jest za jednego z '
                        'Wieszczów Narodowych. Jego dzieła mocno oddziaływały '
                        'na polską kulturę i tożsamość narodową.',
             'role': 'assistant'},
 'model': 'SpeakLeash/bielik-7b-instruct-v0.1-gguf',
 'prompt_eval_count': 41,
 'prompt_eval_duration': 20510000,
 'total_duration': 2652471217}


In [ ]:
import requests

response = requests.post(URL, json=PAYLOAD)
pprint(response.json())

{'created_at': '2024-08-22T15:50:22.192379581Z',
 'done': True,
 'done_reason': 'stop',
 'eval_count': 57,
 'eval_duration': 488890000,
 'load_duration': 4256853,
 'message': {'content': ' Adam Mickiewicz to polski poeta, publicysta i '
                        'filozof, uważany za jednego z Wieszczów Narodowych. '
                        'Jest znany przede wszystkim jako autor epopei "Pan '
                        'Tadeusz".',
             'role': 'assistant'},
 'model': 'SpeakLeash/bielik-7b-instruct-v0.1-gguf',
 'prompt_eval_count': 41,
 'prompt_eval_duration': 8792000,
 'total_duration': 646232811}


## Podsumowanie

Świetnie, w tym tutorialu pokazano jak w podstawowy sposób uruchomić LLM na swoim komputerze. Teraz możesz całkowicie za darmo uruchomić prywatnie LLM i dowolnie z niego korzystać. W przypadku trudniejszych zadań odsyłamy do dokumentacji gdzie będą pokazane bardziej zaawansowane możliwości które z pewnością spełnią bardziej zaawansowane wymagania.